In [1]:
from scipy import stats
import random
import time
import matplotlib.pyplot as plt
import pylob as lob
from IPython.display import clear_output

In [2]:
def generate_orders(n: int):
    result = list()
    
    for _ in range(n):
    
        midprice = 110_000
    
        n_ask_limits = stats.poisson.rvs(1000)
        n_bid_limits = stats.poisson.rvs(1000)
    
        ask_limits_price = stats.expon.rvs(loc=midprice, scale=1, size=n_ask_limits)
        bid_limits_price = -stats.expon.rvs(loc=midprice, scale=1, size=n_bid_limits) + 2*midprice
    
        ask_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_ask_limits)
        bid_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_bid_limits)
    
        ask_limits_params = [lob.OrderParams(lob.OrderSide.ASK, p, q) for (p, q) in zip(ask_limits_price, ask_limits_quantities)]
        bid_limits_params = [lob.OrderParams(lob.OrderSide.BID, p, q) for (p, q) in zip(bid_limits_price, bid_limits_quantities)]
    
        n_markets = stats.poisson.rvs(100)
    
        markets_price = stats.norm.rvs(loc=midprice, scale=2, size=n_markets)
        markets_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_markets)
        markets_bid_or_ask = [random.choice((lob.OrderSide.BID, lob.OrderSide.ASK)) for _ in range(n_markets)]
    
        markets_params = [lob.OrderParams(s, p, q) for (s, p, q) in zip(markets_bid_or_ask, markets_price, markets_quantities)]
    
        orders = ask_limits_params + bid_limits_params + markets_params
        random.shuffle(orders)
        
        result.append(orders)
        
    return result

In [3]:
orders = generate_orders(10)

In [4]:
ob = lob.OrderBook('Simulation')

for o in orders:
    ob.process_many(o)
    print(ob)
    time.sleep(0.7)
    clear_output(wait=True)

   [ORDER-BOOK Simulation]

   ...(506 more asks)
 - Limit(price=110000.11, orders=105, volume=5380.49)
 - Limit(price=110000.10, orders=108, volume=5219.07)
 - Limit(price=110000.09, orders=76, volume=3483.81)
 - Limit(price=110000.08, orders=87, volume=4807.89)
 - Limit(price=110000.07, orders=102, volume=4743.73)
 - Limit(price=110000.06, orders=93, volume=4529.02)
 - Limit(price=110000.05, orders=87, volume=4298.47)
 - Limit(price=110000.04, orders=112, volume=6126.11)
 - Limit(price=110000.03, orders=71, volume=3929.47)
 - Limit(price=110000.02, orders=4, volume=218.31)
 ----------------------------------------------
 - Limit(price=109999.99, orders=1, volume=82.93)
 - Limit(price=109999.98, orders=44, volume=2175.24)
 - Limit(price=109999.97, orders=80, volume=4260.45)
 - Limit(price=109999.96, orders=83, volume=4160.35)
 - Limit(price=109999.95, orders=85, volume=3932.94)
 - Limit(price=109999.94, orders=106, volume=5046.25)
 - Limit(price=109999.93, orders=107, volume=5971.24)
